# Pseudocode

```
Find pivot P
Sort Points by angle (resolve ties in favor of point farther from P)

// Points[1] is the pivot
Stack.push(Points[1]);
Stack.push(Points[2]);
FOR i = 3 TO Points.length
  o <- Cross_product(Stack.second, Stack.top, Points[i]);
  IF o == 0 THEN
    Stack.pop;
    Stack.push(Points[i]);
  ELSEIF o > 0 THEN
    Stack.push(Points[i]);
  ELSE
    WHILE o <= 0 and Stack.length > 2
      Stack.pop;
      o <- Cross_product(Stack.second, Stack.top, Points[i]);
    ENDWHILE
    Stack.push(Points[i]);
  ENDIF
NEXT i

FUNCTION Cross_product(p1, p2, p3)
  RETURN (p2.x - p1.x)*(p3.y - p1.y) - (p3.x - p1.x)*(p2.y - p1.y);
ENDFUNCTION

```

# Generating Case Tests

In [ ]:
import random
import plotly.express as px
import math
import numpy as np
import plotly.graph_objects as go

n = 20 # amount of points
points = np.array([[random.randint(0,n*10), random.randint(0,n*10)] for i in range (0, n)])
points = np.unique(points, axis=0) # removing possible duplicates

fig = go.Figure()
fig.add_trace(go.Scatter(x=points[:,0], y=points[:,1], text=[f"P({x},{y})" for x,y in points],
                    textposition="bottom center",
                    mode='markers+text', name='markers'))
fig.update_layout(autosize=False, width=500, height=500, margin=dict(l=25, r=25, b=25, t=25, pad=2), xaxis_range=[-5,n*10], yaxis_range=[-5,n*10])

Setting start time

In [ ]:
import time
start = time.time()

## Finding Pivot (P0) - O(nlogn)

In [ ]:
y_sorted_points = np.array(sorted(points, key=lambda tup: [tup[1], tup[0]]))
P0 = y_sorted_points[0]
# P0

## Sorting points by their polar angle from P0 - O(nlogn)
Unties in favor of the ones that are more distant to the pivot (P0)

In [ ]:
points_with_angle_and_distance=np.array(
    [
      [
        xi,
        yi,
        abs(math.atan2( P0[1] - yi  ,  xi - P0[0] ) * ( 180 / math.pi )),
        np.linalg.norm(np.array((P0[0], P0[1])) - np.array((xi, yi)))
    ] for xi, yi in y_sorted_points]
)
angle_distance_sorted_points = np.array(sorted(points_with_angle_and_distance, key=lambda tup: [tup[2], tup[3]]))
# angle_distance_sorted_points

## Setting stack object - O(1)

In [ ]:
s = []
s.append(list(angle_distance_sorted_points[0])) #P0
if len(angle_distance_sorted_points) > 1: s.append(list(angle_distance_sorted_points[1]))
# s

## Defining cross production function - O(1)

In [ ]:
def cross_product(p1, p2, p3):
  # (p2.x - p1.x)*(p3.y - p1.y) - (p3.x - p1.x)*(p2.y - p1.y)
  return (p2[0] - p1[0])*(p3[1] - p1[1]) - (p3[0] - p1[0])*(p2[1] - p1[1])

## Applying Graham's Scan - O(n)

In [ ]:
for i in range(2, len(angle_distance_sorted_points)):
  o = cross_product(s[-2], s[-1], angle_distance_sorted_points[i])
  if o == 0:
    s.pop()
    s.append(list(angle_distance_sorted_points[i]))
  elif o > 0:
    s.append(list(angle_distance_sorted_points[i]))
  else:
    while o <= 0 and len(s) > 2:
      s.pop()
      o = cross_product(s[-2], s[-1], angle_distance_sorted_points[i])
    s.append(list(angle_distance_sorted_points[i]))
s.append(list(angle_distance_sorted_points[0]))

Calculating execution time

In [ ]:
end = time.time()
print(end - start)

# Plotting graphics displaying the results

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=points[:,0], y=points[:,1], text=[f"P({x},{y})" for x,y in points],
                    textposition="bottom center",
                    mode='markers+text', name='internal points'))
s = np.array(s)
fig.add_trace(go.Scatter(x=s[:,0], y=s[:,1],
                    mode='lines+markers',
                    name='hull convex points'))
fig.update_layout(autosize=False, width=650, height=500, margin=dict(l=25, r=25, b=25, t=25, pad=2), xaxis_range=[-5,n*10], yaxis_range=[-5,n*10])